In [37]:
import torch
from torch import nn
import torchvision
from torchvision import transforms
from torch.utils.data import Dataset
from torchvision.models import MobileNet_V3_Small_Weights
from torchvision.models.mobilenetv3 import mobilenet_v3_small

from PIL import Image
import os

from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np

import time
from tqdm import tqdm

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
!unzip "/content/drive/MyDrive/train.zip"

Выходные данные были обрезаны до нескольких последних строк (5000).
  inflating: train/ce8698ba49e041ecb1818078d3b9710e.jpg  
  inflating: train/368153b6dd4d48648df6bf12249822e1.jpg  
  inflating: train/9c0b9afae616496f9f459fa3b08d8bb2.jpg  
  inflating: train/e712073f78044426848ccadaa9a42617.jpg  
  inflating: train/80fe4b0ceb584d7f81211b035301be6c.jpg  
  inflating: train/c891bd23a94440f886356cb670d0d9c8.jpg  
  inflating: train/78dd3672674b4cb09758633fbba0b9c8.jpg  
  inflating: train/0d287d361c5c4ce9bd30821771a4d2c8.jpg  
  inflating: train/2a8e9675ed0a46a992d4f338ac781bf9.jpg  
  inflating: train/1e57c18543a249109b0d351410927abb.jpg  
  inflating: train/5c84a22bd3d542c59ceaf5975318b20a.jpg  
  inflating: train/b4f1f7f605f7498eb2e62357f638e7e0.webp  
  inflating: train/13e7a59be12b4618befcb7e50807c5bd.jpg  
  inflating: train/a2ab1ec52b844bc9a0b1b70f9750546c.jpg  
  inflating: train/88f043b30919463ea42752e043b4e4f8.jpg  
  inflating: train/3647178cb6294d91b34bb2cfe55f925c.webp  
  

In [4]:
def set_requires_grad(model, value=False):
    for param in model.parameters():
        param.requires_grad = value

In [52]:
def train_model(model, dataloaders, criterion, optimizer,
                phases, num_epochs=3):

    scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer, gamma=0.9, verbose=True)
    start_time = time.time()

    acc_history = {k: list() for k in phases}
    loss_history = {k: list() for k in phases}

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in phases:
            if phase == 'train':
                model.train()  # Set model to training mode
            else:
                model.eval()  # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0

            # Iterate over data.
            n_batches = len(dataloaders[phase])
            for inputs, labels in tqdm(dataloaders[phase], total=n_batches):
                inputs = inputs.to(device)
                labels = labels.to(device)

                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    loss = criterion(outputs, labels)

                    _, preds = torch.max(outputs, 1)

                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # statistics
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)

            if phase == 'train':
              scheduler.step()

            epoch_loss = running_loss / len(dataloaders[phase].dataset)
            epoch_acc = running_corrects.double()
            epoch_acc /= len(dataloaders[phase].dataset)

            print('{} Loss: {:.4f} Acc: {:.4f}'.format(phase, epoch_loss,
                                                       epoch_acc))
            loss_history[phase].append(epoch_loss)
            acc_history[phase].append(epoch_acc)

        print()

    time_elapsed = time.time() - start_time
    print('Training complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60,
                                                        time_elapsed % 60))

    return model, acc_history

In [45]:
model = torchvision.models.mobilenetv3.mobilenet_v3_small(pretrained=True)
set_requires_grad(model, False)
set_requires_grad(model.classifier, True)
model.classifier[3] = nn.Sequential(
    nn.Linear(in_features=1024, out_features=35)
)
model
# torchvision.models.MobileNet_V3_Small_Weights.IMAGENET1K_V1.transforms()
model = None

In [59]:
def init_model(device, num_classes):
    model = torchvision.models.mobilenetv3.mobilenet_v3_small(weights=MobileNet_V3_Small_Weights.IMAGENET1K_V1)
    set_requires_grad(model, False)
    set_requires_grad(model.classifier, True)
    model.classifier[3] = nn.Sequential(
    nn.Linear(in_features=1024, out_features=35)
)
    model = model.to(device)
    return model


In [29]:
class ArtDataset(Dataset):
    def __init__(self, root_dir, csv_path=None, transform=None):

        self.transform = transform
        self.files = [os.path.join(root_dir, fname) for fname in os.listdir(root_dir)]
        self.targets = None
        if csv_path:
            df = pd.read_csv(csv_path, sep="\t")
            self.targets = df["label_id"].tolist()
            self.files = [os.path.join(root_dir, fname) for fname in df["image_name"].tolist()]

    def __len__(self):
        return len(self.files)

    def __getitem__(self, idx):
        image = Image.open(self.files[idx]).convert('RGB')
        target = self.targets[idx] if self.targets else -1
        if self.transform:
            image = self.transform(image)
        return image, target

In [20]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


In [34]:
#if __name__ == "__main__":
def main():
    img_size = 224

    trans = transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
        ])


    dset = ArtDataset(TRAIN_DATASET, TRAIN_CSV, trans)
    labels = dset.targets
    indices = list(range(len(labels)))
    ind_train, ind_test, _, _ = train_test_split(indices, labels, test_size=0.2, random_state=139, stratify=labels)

    trainset = torch.utils.data.Subset(dset, ind_train)
    testset = torch.utils.data.Subset(dset, ind_test)

    batch_size = 120
    num_workers = 2
    trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size,
                                              shuffle=True, num_workers=num_workers)

    testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size,
                                             shuffle=False, num_workers=num_workers)

    loaders = {'train': trainloader, 'val': testloader}

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    model = init_model(device, num_classes=35)

    # pretrain_optimizer = torch.optim.Adam(params=model.parameters(), #.classifier[3].parameters(),
    #                                      lr=0.001)

    train_optimizer = torch.optim.Adam(model.parameters(), lr=0.005)

    criterion = nn.CrossEntropyLoss()

    # Pretrain
    # запустить предобучение модели на две эпохи
    # pretrain_results = train_model(model, loaders, criterion, pretrain_optimizer,
    #                                phases=['train', 'val'], num_epochs=3)

    # Train
    # запустить дообучение модели
    set_requires_grad(model, True)
    train_results = train_model(model, loaders, criterion, train_optimizer,
                                phases=['train', 'val'], num_epochs=15)

    torch.save(model.state_dict(), MODEL_WEIGHTS)

In [60]:
# hardcode
MODEL_WEIGHTS = "/content/drive/MyDrive/baseline.pt"
TRAIN_DATASET = "/content/train"
TRAIN_CSV = "/content/drive/MyDrive/train.csv"

main()

Adjusting learning rate of group 0 to 5.0000e-03.
Epoch 0/14
----------


 45%|████▍     | 26/58 [00:57<01:03,  1.98s/it]/usr/local/lib/python3.10/dist-packages/PIL/Image.py:996: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(
100%|██████████| 58/58 [02:06<00:00,  2.18s/it]


Adjusting learning rate of group 0 to 4.5000e-03.
train Loss: 1.8530 Acc: 0.4631


100%|██████████| 15/15 [00:31<00:00,  2.11s/it]


val Loss: 14.5704 Acc: 0.0905

Epoch 1/14
----------


...

val Loss: 1.5715 Acc: 0.7278

Epoch 14/14
----------


 17%|█▋        | 10/58 [00:23<01:40,  2.09s/it]/usr/local/lib/python3.10/dist-packages/PIL/Image.py:996: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(
100%|██████████| 58/58 [02:11<00:00,  2.27s/it]


Adjusting learning rate of group 0 to 1.0295e-03.
train Loss: 0.0026 Acc: 0.9997


100%|██████████| 15/15 [00:33<00:00,  2.22s/it]


val Loss: 1.5804 Acc: 0.7208

Training complete in 40m 11s


In [61]:
def make_submission():
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = init_model(device, num_classes=35)
    model.load_state_dict(torch.load(MODEL_WEIGHTS))
    model.eval()

    trans = transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
        ])

    dset = ArtDataset(TEST_DATASET, transform=trans)
    batch_size = 16
    num_workers = 4
    testloader = torch.utils.data.DataLoader(dset, batch_size=batch_size,
                                            shuffle=False, num_workers=num_workers)

    all_image_names = [item.split("/")[-1] for item in dset.files]
    all_preds = []
    model = model.eval()
    with torch.no_grad():
        for idx, (images, _) in enumerate(testloader, 0):
            images = images.to(device)
            outputs = model(images)
            _, preds = torch.max(outputs, 1)
            all_preds.extend(preds.cpu().numpy().tolist())

    with open(SUBMISSION_PATH, "w") as f:
        f.write("image_name\tlabel_id\n")
        for name, cl_id in zip(all_image_names, all_preds):
            f.write(f"{name}\t{cl_id}\n")

In [62]:
MODEL_WEIGHTS = "/content/drive/MyDrive/baseline.pt"
TEST_DATASET = "/content/train/"
SUBMISSION_PATH = "/content/drive/MyDrive/submission.csv"

In [63]:
make_submission()

/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:557: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(
/usr/local/lib/python3.10/dist-packages/PIL/Image.py:996: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(
